In [1]:
import pandas as pd
import numpy as np
import torch as th
from torchvision import datasets, transforms
import syft as sy

In [2]:
duet = sy.launch_duet()

🎤  🎸  ♪♪♪ Starting Duet ♫♫♫  🎻  🎹

♫♫♫ > DISCLAIMER: Duet is an experimental feature currently in beta.
♫♫♫ > Use at your own risk.


    > ❤️ Love Duet? Please consider supporting our community!
    > https://github.com/sponsors/OpenMined

♫♫♫ > Punching through firewall to OpenGrid Network Node at:
♫♫♫ > http://ec2-18-218-7-180.us-east-2.compute.amazonaws.com:5000
♫♫♫ >
♫♫♫ > ...waiting for response from OpenGrid Network... 
♫♫♫ > DONE!
♫♫♫ > Duet Server ID: e0cc42162af979e38097fe5e3007d4fe

♫♫♫ > STEP 1: Send the following code to your Duet Partner!

import syft as sy
duet = sy.duet("e0cc42162af979e38097fe5e3007d4fe")

♫♫♫ > STEP 2: Ask your partner for their Client ID and enter it below!
♫♫♫ > Duet Partner's Client ID: 780f8eb25fb6683a0b80506d7656ed7b

♫♫♫ > Connecting...


C:\Users\davit\anaconda3\envs\Final_Project\lib\site-packages\aiortc\rtcdtlstransport.py:211: CryptographyDeprecationWarning: This version of cryptography contains a temporary pyOpenSSL fallback path. Upgrade pyOpenSSL now.
  _openssl_assert(lib.SSL_CTX_use_certificate(ctx, self._cert._x509) == 1)  # type: ignore
C:\Users\davit\anaconda3\envs\Final_Project\lib\site-packages\aiortc\rtcdtlstransport.py:186: CryptographyDeprecationWarning: This version of cryptography contains a temporary pyOpenSSL fallback path. Upgrade pyOpenSSL now.
  value=certificate_digest(self._cert._x509),  # type: ignore



♫♫♫ > CONNECTED!



In [3]:
PIL_to_Tensor = transforms.Compose([
                transforms.Resize(size = (256,256)),
                transforms.RandomVerticalFlip(0.1),
                transforms.RandomHorizontalFlip(0.1),
                transforms.RandomRotation(degrees = 10),
                transforms.CenterCrop(size=224),
                transforms.ToTensor(),
                transforms.Normalize([0.485, 0.456, 0.406],[0.229, 0.224, 0.225])
    ])

In [4]:
# Divide the data into 3 sets to represent the three hospitals.
normal_train = datasets.ImageFolder('chest_xray/train/_normal', transform= PIL_to_Tensor)

In [5]:
normal_train

Dataset ImageFolder
    Number of datapoints: 1341
    Root location: chest_xray/train/_normal
    StandardTransform
Transform: Compose(
               Resize(size=(256, 256), interpolation=bilinear)
               RandomVerticalFlip(p=0.1)
               RandomHorizontalFlip(p=0.1)
               RandomRotation(degrees=[-10.0, 10.0], interpolation=nearest, expand=False, fill=0)
               CenterCrop(size=(224, 224))
               ToTensor()
               Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
           )

In [6]:
pneumonia_train = datasets.ImageFolder('chest_xray/train/_pneumonia', transform= PIL_to_Tensor)

In [7]:
pneumonia_train

Dataset ImageFolder
    Number of datapoints: 3875
    Root location: chest_xray/train/_pneumonia
    StandardTransform
Transform: Compose(
               Resize(size=(256, 256), interpolation=bilinear)
               RandomVerticalFlip(p=0.1)
               RandomHorizontalFlip(p=0.1)
               RandomRotation(degrees=[-10.0, 10.0], interpolation=nearest, expand=False, fill=0)
               CenterCrop(size=(224, 224))
               ToTensor()
               Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
           )

In [8]:
X = []
Y = []

In [9]:
for index in range(200, 300):
    normal_image = normal_train[index][0].type(th.float32).numpy()
    X.append(normal_image)
    Y.append(0)

In [10]:
for index in range(200, 300):
    pneumonia_image = pneumonia_train[index][0].type(th.float32).numpy()
    X.append(pneumonia_image)
    Y.append(1)

In [11]:
X = th.FloatTensor(X)
Y = th.FloatTensor(Y)

In [12]:
X = X.tag("#X")
Y = Y.tag("#Y")

In [13]:
X_ptr = X.send(duet, pointable=True)
Y_ptr = Y.send(duet, pointable=True)

In [14]:
duet.store.pandas

,ID,Tags,Description,object_type
0,<UID: 4c10cbc524c24e1bba806eb20dfe4865>,[#X],,<class 'torch.Tensor'>
1,<UID: 7dd0441e837e422abe49ca6fb0ecfd57>,[#Y],,<class 'torch.Tensor'>


In [15]:
duet.requests

[]

In [16]:
# Make it so we accept those request from the DS
duet.requests.add_handler(action="accept")